In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/09 20:06:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [5]:
df_green = df_green \
            .withColumnRenamed('lpep_pickup_datetime','pickup_datetime') \
            .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')


In [6]:
df_yellow = df_yellow \
            .withColumnRenamed('tpep_pickup_datetime','pickup_datetime') \
            .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

### 2: Finding common columns

In [7]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [8]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [9]:
print(common_colums)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


In [10]:
df_green.select(common_colums).show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|       1|2020-01-21 08:09:42|2020-01-21 08:24:52|                 N|         1|           7|          82|              1|          2.9|       14.0|  0.0|    0.5|       0.0|         0.0|                  0.3|        14.8|           

###  Adding service types to identify dataset

In [11]:
from pyspark.sql import functions as F

In [12]:
df_green_sel = df_green  \
        .select(common_colums)  \
        .withColumn('service_type',F.lit('green'))

In [13]:
df_yellow_sel = df_yellow  \
        .select(common_colums)  \
        .withColumn('service_type',F.lit('yellow'))

### 3: Combining yellow and green data

In [14]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [15]:
df_trips_data.groupBy('service_type').count().show()

[Stage 3:======================================================>  (24 + 1) / 25]

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



### 4: Querying with SQL


#### Now we can see how to use SQL for querying this data. First, we need to tell Spark that this DataFrame is a table. For that, we use:



In [19]:
df_trips_data.registerTempTable('trips_data')

/opt/conda/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
spark.sql("""
        SELECT * FROM trips_data LIMIT 10;
        """ ).show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       1|2020-01-21 08:09:42|2020-01-21 08:24:52|                 N|         1|           7|          82|              1|          2.9|       14.0|  0.0|    0.5|       0.0|         0.0|       

In [24]:

spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

[Stage 14:=====================================================>  (24 + 1) / 25]

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



#### Now let's execute this query from module 4:

In [27]:

df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [28]:
df_result \
    .select('revenue_zone', 'revenue_month', 'service_type', 'revenue_monthly_total_amount', 'avg_montly_passenger_count') \
    .show()

[Stage 17:=====================================================>  (24 + 1) / 25]

+------------+-------------------+------------+----------------------------+--------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_total_amount|avg_montly_passenger_count|
+------------+-------------------+------------+----------------------------+--------------------------+
|         218|2020-01-01 00:00:00|       green|           27375.14999999997|        1.0754716981132075|
|          85|2020-01-01 00:00:00|       green|          22320.909999999938|        1.3349753694581281|
|         116|2019-12-01 00:00:00|       green|          29.439999999999998|                       1.0|
|          93|2020-02-01 00:00:00|       green|                     4123.47|         1.380952380952381|
|         143|2020-02-01 00:00:00|       green|          1640.8499999999997|                       1.0|
|         225|2020-02-01 00:00:00|       green|          47254.729999999836|        1.1892332789559543|
|         112|2020-01-01 00:00:00|       green|           27470.

#### 5: Saving the result

In [29]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')